In [1]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Activation, Dense, Dropout
from keras.layers import concatenate

import numpy as np
import tensorflow as tf

Using TensorFlow backend.


## define parameters

In [2]:
N = 5
M = 3
K = 2
N_u = 5
M_v = 6

## define a_u and a_v as the tensor, because we are not sampling anything

In [3]:
X = np.random.randn(N,M)
a_u = np.random.randn(N,N_u)
a_v = np.random.randn(M,M_v)

# X = tf.convert_to_tensor(X, dtype=tf.float32)
a_u = tf.convert_to_tensor(a_u, dtype=tf.float32)
a_v = tf.convert_to_tensor(a_v, dtype=tf.float32)


def next_batch(batch_size):
    '''
    returnn data of shape [ [batch,N], [batch,M], [batch,1] ]
    fetch index from the matrix of shape [batch,batch]
    '''
    x_idx = np.random.randint(N,size=(batch_size))
    y_idx = np.random.randint(M,size=(batch_size))
    
    rating_label = []
    for i,j in zip(x_idx,y_idx):
        rating_label.append(X[i][j])
#     rating_label = tf.convert_to_tensor(rating_label,dtype=tf.float32)
#     rating_label = tf.reshape(rating_label,shape=[-1,1])
    
#     return Xn, Xm, rating_label
    return x_idx, y_idx, rating_label

## build model

In [4]:
input_user = tf.placeholder(tf.int32,shape=(None,),name='batch-user')
input_user_hot = tf.one_hot(input_user,N, name='one-hot-user')
user_latent = Dense(K,activation='linear', name='user-latent')(input_user_hot)

input_item = tf.placeholder(tf.int32,shape=(None,),name='batch-item')
input_item_hot = tf.one_hot(input_item,M, name='one-hot-item')
item_latent = Dense(K,activation='linear', name='item-latent')(input_item_hot)

info_user = tf.matmul(input_user_hot, a_u)
info_user_latent = Dense(K,activation='linear',name='info-user-latent')(info_user)

info_item = tf.matmul(input_item_hot, a_v)
info_item_latent = Dense(K,activation='linear', name='item-user-latent')(info_item)

In [5]:
input_user, user_latent, input_item, item_latent, info_user_latent, info_item_latent

(<tf.Tensor 'batch-user:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'user-latent/BiasAdd:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'batch-item:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'item-latent/BiasAdd:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'info-user-latent/BiasAdd:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'item-user-latent/BiasAdd:0' shape=(?, 2) dtype=float32>)

In [6]:
pred_rating = tf.matmul(user_latent,tf.transpose(item_latent))#,transpose_item_latent)
pred_rating = tf.diag_part(pred_rating)
pred_rating = tf.reshape(pred_rating,shape=[-1,1])

pred_rating_content = tf.matmul(info_user_latent,tf.transpose(info_item_latent))#,transpose_info_item_latent)
pred_rating_content = tf.diag_part(pred_rating_content)
pred_rating_content = tf.reshape(pred_rating_content, shape=[-1,1])

final_rating = pred_rating + pred_rating_content

In [7]:
final_rating

<tf.Tensor 'add:0' shape=(?, 1) dtype=float32>

## loss function (mse. no classification)

In [8]:
actual = tf.placeholder(tf.float32, shape=(None,),name='rating')

loss = tf.reduce_mean(tf.square(pred_rating - actual))
loss_user = tf.reduce_mean(tf.square(user_latent - info_user_latent))
loss_item = tf.reduce_mean(tf.square(item_latent - info_item_latent))
final_loss = loss + loss_user + loss_item

In [9]:
final_loss

<tf.Tensor 'add_2:0' shape=() dtype=float32>

In [10]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(final_loss)

correct_pred = tf.equal(final_rating, actual)
correct_pred = tf.cast(correct_pred, tf.float32)
accuracy = tf.reduce_mean(correct_pred)

accuracy

<tf.Tensor 'Mean_3:0' shape=() dtype=float32>

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 2
for i in range(10):
    batch = next_batch(batch_size)
    feed_data = {input_user : batch[0], input_item : batch[1], actual : batch[2]}
#     if i % 2 == 0:
#         train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
#         print('step %d, training accuracy %g' % (i, train_accuracy))
        
    loss_iter = sess.run(final_loss,feed_dict=feed_data)
    print('step %d, training accuracy %g' % (i, loss_iter))

    sess.run(optimizer, feed_dict=feed_data)

step 0, training accuracy 2.66976
step 1, training accuracy 2.59278
step 2, training accuracy 2.64909
step 3, training accuracy 3.25894
step 4, training accuracy 2.7083
step 5, training accuracy 3.10092
step 6, training accuracy 2.04663
step 7, training accuracy 2.40559
step 8, training accuracy 1.98688
step 9, training accuracy 4.36548
